<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/report_metr_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import zipfile
from collections import namedtuple

In [2]:
data = pd.read_excel('/content/Для анализа.xlsx', index_col='№ п/п')
data.head()

,Дата получения документа,Дата возврата документа,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры)
№ п/п,,,,,,,,,,
1,2021-12-28,2021-12-29,ИТКС.1114-21,ИТКС.443124.001 УБР-1,"ОРС ТВГ, Малюгин",3,Докторова,NaN,NaN,NaN
2,2021-12-28,2021-12-29,ИТКС.685669.001СБ,Жгут,"ОРС ТВГ, Малюгин",4,Докторова,NaN,"10, 17",NaN
3,2021-12-28,2021-12-29,ИТКС.443124.001ИЭ,NaN,"ОРС ТВГ, Бурков",9,Докторова,NaN,"2, 19, 3, 17, 6, 7","17, 4"
4,2021-12-28,2021-12-29,ИТКС.443124.001ПЭ4,NaN,"ОРС ТВГ, Малюгин",1,Докторова,NaN,NaN,NaN
5,2021-12-28,2021-12-29,ИТКС.443124.001Е1,NaN,"ОРС ТВГ, Малюгин",2,Докторова,NaN,17,"17, 3"


# 1 Обработка пропусков в данных

In [3]:
# заполняем пропуски в столбце 'Отдел, разработчик' значениями предыдущей строки
data['Отдел, разработчик'] = data['Отдел, разработчик'].fillna(method='ffill')

In [4]:
data['Дата получения документа'] = data['Дата получения документа'].fillna(method='ffill')
data['Дата возврата документа'] = data['Дата возврата документа'].fillna(data['Дата получения документа'])

In [5]:
data[' Замечания (шифры)'] = data[' Замечания (шифры)'].fillna(0)
data['Повторные замечания (шифры)'] = data['Повторные замечания (шифры)'].fillna(0)

# 2 Создание дополнительных столбцов

In [6]:
data['Срок проверки'] = data['Дата возврата документа'] - data['Дата получения документа'] + pd.offsets.Day(1)

In [7]:
data['Разработчик'] = ''

In [8]:
# разделяем значения из столбца 'Отдел, разработчик' на 2 значения
for i in range(data['Отдел, разработчик'].shape[0]):
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
  for s in data['Отдел, разработчик'].iloc[i]:
    s = s.strip()
    data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
# исправляем типичные ошибки в названиях отделов + актуализируем названия
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРМД', 'ОРМС'],'ОРМ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРЛГ', 'ОРИС')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСТВГ', 'ОРС ТВГ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСДВ', 'ОРС ДВ')

data['Разработчик'] = data['Разработчик'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')

In [10]:
# добавляем к фамилии разработчика суффикс с названием его отдела
for i in range(data['Разработчик'].shape[0]):
  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]

# 3 Обработка столбца с замечаниями

In [11]:
# для удобства дальнейшей обработки создаем список шифров
for i in range(data[' Замечания (шифры)'].shape[0]):
   a = [s.strip() for s in re.split('[,.]', str(data[' Замечания (шифры)'].iloc[i]))]
   data[' Замечания (шифры)'].iloc[i] = a

for i in range(data['Повторные замечания (шифры)'].shape[0]):
   a = [s.strip() for s in re.split('[,.]', str(data['Повторные замечания (шифры)'].iloc[i]))]
   data['Повторные замечания (шифры)'].iloc[i] = a

In [12]:
data.sample(10)

,Дата получения документа,Дата возврата документа,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры),Срок проверки,Разработчик
№ п/п,,,,,,,,,,,,
84,2022-01-18,2022-01-19,ИТКС.301313.042СБ,Основание,КО,8,Авилов,NaN,[0],[0],2 days,Хохлов_КО
1196,2022-07-12,2022-07-13,ИТКС.711162.003,Гайка,КО,1,Авилов,NaN,"[21, 24]",[0],2 days,Бауков_КО
2269,2022-11-23,2022-11-23,ОГТ.112-22,Акселерометр А-18,ОГТ,1,Сидорова,NaN,[0],[0],1 days,Бушуева_ОГТ
1184,2022-07-12,2022-07-13,ИТКС.301318.073СБ,Подставка,КО,4,Авилов,NaN,[29],[0],2 days,Бауков_КО
2335,2022-11-30,2022-11-30,ИТКС.301318.081CБ,Присп. для испытаний БИНС-5 на виб. И удар. Ст...,КО,2,Авилов,NaN,[0],[0],1 days,Плигин_КО
1530,2022-09-05,2022-09-05,ИТКС.754312.054,Знак заводской,ОРИС,2,Марусина,NaN,[0],[0],1 days,Мотов_ОРИС
42,2022-01-12,2022-01-12,ИТКС.878-21,ИТКС.461532.034 БИНС-РС,ОРИС,1,Иванов,NaN,[0],[0],1 days,Колесников_ОРИС
2024,2022-12-01,2022-12-01,ИТКС.637 2/2-22,ИТКС.461532.039 БИНС-20,КО,1,Марусина,Присвоение литеры О,[0],[0],1 days,Миннеахметова_КО
874,2022-05-31,2022-05-31,СЯПИ.433724.013-02 ЭТ,Генератор ГМ-2-2,ОРИС,1,Иванов,отраб. на литер,[0],[0],1 days,Епифанова_ОРИС


In [13]:
# функция для подсчета количества ошибок по отделам

def count_codes(data: pd.DataFrame, department: str) -> dict:
  # словарь для хранения количества замечаний по каждому шифру
  codes = dict(zip(np.arange(0, 31), np.zeros(30)))

  # цикл по строкам датафрейма  с помощью указателя i
  for i in range(data.shape[0]):
    # если строка относится к нужному отделу
    if data['Отдел, разработчик'].iloc[i] == department:
      # берем значение из столбца с замечаниями
      
      for c in data[' Замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров
        codes[int(c)] += 1
      for c in data['Повторные замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров за исключением 0 шифра
        if int(c) != 0:
          codes[int(c)] += 1
  
  return codes

In [14]:
# функция для подсчета количества ошибок по разработчику

def count_codes_by_person(data: pd.DataFrame, person: str) -> dict:
  # словарь для хранения количества замечаний по каждому шифру
  codes = dict(zip(np.arange(0, 31), np.zeros(30)))

  # цикл по строкам датафрейма  с помощью указателя i
  for i in range(data.shape[0]):
    # если строка относится к нужному разработчику
    if data['Разработчик'].iloc[i] == person:
      # берем значение из столбца с замечаниями
      
      for c in data[' Замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров
        codes[int(c)] += 1
      for c in data['Повторные замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров за исключением 0 шифра
        if int(c) != 0:
          codes[int(c)] += 1
  
  return codes

# 4 Формирование отчетов

In [15]:
def make_main_report(data: pd.DataFrame) -> pd.DataFrame:
  report = pd.DataFrame(columns=['Проверено документов, шт',
                                   'Проверено листов А4, шт',
                                   'Всего замечаний, шт',
                                   'Доля документов с замечаниями, %',
                                   'Количество замечаний на 100 листов А4, шт'])
  report['Проверено документов, шт'] = data.groupby(['Отдел, разработчик']).count()['Дата получения документа']
  report['Проверено листов А4, шт'] = data.groupby(['Отдел, разработчик']).sum()

  # в цикле создаем вспомогательные переменные для подсчета замечаний и показателей
  for name in report.index.to_list():
    # создаем словарь для каждого отдела с количеством замечаний
    codes = count_codes(data, name)
    # переменная для хранения количества замечаний без учета шифра '17'
    total_without_17 = 0
    # переменная для хранения количества замечаний с шифром '17'
    count_17 = codes[17]
    # переменная для хранения количества документов без замечаний
    without_mistakes = codes[0]
    for c in codes.keys():
      if c not in (0, 17):
        total_without_17 += codes[c]
    # переменная для хранения общего числа замечаний
    total_count = total_without_17 + count_17
    
    report.loc[name, 'Всего замечаний, шт'] = total_count
    report.loc[name, 'Доля документов с замечаниями, %'] = np.round(((1 - (without_mistakes / report.loc[name, 'Проверено документов, шт'])) * 100), 1)
    report.loc[name, 'Количество замечаний на 100 листов А4, шт'] = np.round(((total_count / report.loc[name, 'Проверено листов А4, шт']) * 100), 1)

  # создаем переменные для хранения сводных показателей по предприятию
  all_docs = np.sum(report['Проверено документов, шт'])
  all_pages = np.sum(report['Проверено листов А4, шт'])
  all_mistakes = np.sum(report['Всего замечаний, шт'])
  all_docs_with_mistakes = np.sum(report['Проверено документов, шт'] * (report['Доля документов с замечаниями, %'] / 100))
  
  # добавляем в датафрейм строку со сводными показателями
  report.loc['Общая статистика по предприятию'] = [all_docs, all_pages, all_mistakes, np.round((all_docs_with_mistakes / all_docs)*100, 1), np.round((all_mistakes / all_pages)*100, 1)]

  return report

In [16]:
def make_codes_report(data: pd.DataFrame) -> pd.DataFrame:
  Codes = namedtuple('Codes', ['department', 'codes'])
  counts_of_codes = [Codes(dp, count_codes(data, dp)) for dp in data['Отдел, разработчик'].unique()]

  cols = list(data['Отдел, разработчик'].unique())
  cols.insert(0, 'Шифр')
  report = pd.DataFrame(columns=[cols])

  for nt in counts_of_codes:
    report[nt.department] = nt.codes.values()
  
  report['Шифр'] = report.index
  
  return report

In [17]:
def make_codes_by_person_report(data: pd.DataFrame) -> pd.DataFrame:
  Codes = namedtuple('Codes', ['person', 'codes'])
  counts_of_codes = [Codes(pers, count_codes_by_person(data, pers)) for pers in data['Разработчик'].unique()]

  cols = list(data['Разработчик'].unique())
  cols.insert(0, 'Шифр')
  report = pd.DataFrame(columns=[cols])

  for nt in counts_of_codes:
    report[nt.person] = nt.codes.values()
  
  report['Шифр'] = report.index
  report = report.T
  report = report.drop('Шифр', axis=0)

  report.insert(0, 'Отдел', [report.index.values[i][0].split('_')[1] for i in range(report.index.shape[0])])

  return report

In [18]:
def make_time_report(data: pd.DataFrame) -> pd.DataFrame:
  report = pd.DataFrame(data['Срок проверки'].value_counts())
  return report

# 5 Деление датасета на части

In [19]:
# разделим датасет на части по кварталам и полугодиям
data_1q = data[(data['Дата получения документа'] > '2022-01-01') & (data['Дата получения документа'] < '2022-04-01')]
data_2q = data[(data['Дата получения документа'] >= '2022-04-01') & (data['Дата получения документа'] < '2022-07-01')]
data_3q = data[(data['Дата получения документа'] >= '2022-07-01') & (data['Дата получения документа'] < '2022-10-01')]
data_4q = data[(data['Дата получения документа'] >= '2022-10-01') & (data['Дата получения документа'] < '2023-01-01')]
data_1_half_year = data[(data['Дата получения документа'] >= '2022-01-01') & (data['Дата получения документа'] < '2022-07-01')]
data_2_half_year = data[(data['Дата получения документа'] >= '2022-07-01') & (data['Дата получения документа'] < '2023-01-01')]
data_full = data[(data['Дата получения документа'] >= '2022-01-01') & (data['Дата получения документа'] < '2023-01-01')]
data_list = [data_1q, data_2q, data_3q, data_4q, data_1_half_year, data_2_half_year, data_full]

In [20]:
# создадим аттрибуты с именем датасета
data_1q.name = '1_квартал'
data_2q.name = '2_квартал'
data_3q.name = '3_квартал'
data_4q.name = '4_квартал'
data_1_half_year.name = '1_полугодие'
data_2_half_year.name = '2_полугодие'
data_full.name = 'Весь_год'

In [21]:
print(f'1 квартал: {data_1q.shape} \n2 квартал: {data_2q.shape} \n3 квартал: {data_3q.shape}  \n4 квартал: {data_4q.shape} \n1 полугодие: {data_1_half_year.shape}  \n2 полугодие: {data_2_half_year.shape} \nВесь год: {data_full.shape}')

1 квартал: (493, 12) 
2 квартал: (569, 12) 
3 квартал: (722, 12)  
4 квартал: (929, 12) 
1 полугодие: (1062, 12)  
2 полугодие: (1651, 12) 
Весь год: (2713, 12)


# 6 Сохранение отчетов

In [22]:
for d in data_list:
  with zipfile.ZipFile(f'{d.name}.zip', 'w') as zf:
    with zf.open(f'{d.name}_основной_отчет.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_main_report(d)
        df.to_excel(writer)  

    with zf.open(f'{d.name}_основной_по_ошибкам.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_codes_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_ошибки_по_разработчику.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_codes_by_person_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_основной_по_срокам.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_time_report(d)
        df.to_excel(writer)
    

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
